In [1]:
import pandas as pd
import os, pickle
import numpy as np
import pymongo

import sys
sys.path.append('../')
import src.IQ as IQ


myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/admin")
BLE = myclient["BLE"]

def query(collection, filter:dict, addFrameColumn=True):
    df =  pd.DataFrame(list(collection.find(filter)))
    if addFrameColumn:
        df['frame'] = df.apply(lambda x: x['I'] + np.dot(x['Q'],1j), axis=1)
    return df.copy()

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.backend import clear_session
from tensorflow.keras.regularizers import l1, l2
import threading
from sklearn.preprocessing import MinMaxScaler





# We will start with a size that is approximately half of the input size
def run(encoding_dim,input_tensor_features,output_tensor_features, plot = False):
    tf.keras.backend.clear_session()
    input_shape = input_tensor_features[0].shape[0]
    output_shape = output_tensor_features[0].shape[0]
    input_data = Input(shape=(input_shape,))
    encoded = Dense(encoding_dim + (input_shape - encoding_dim )//2, activation='relu')(input_data)
    encoded = Dense(encoding_dim + (input_shape - encoding_dim )//4, activation='tanh')(encoded)
    
    encoded = Dense(encoding_dim, activation='linear')(encoded) # kernel_regularizer=l1(0.01)


    decoded = Dense(encoding_dim + (output_shape - encoding_dim )//4, activation='tanh')(encoded) 
    decoded = Dense(encoding_dim + (output_shape - encoding_dim )//2, activation='relu')(decoded)
    decoded = Dense(output_shape, activation='linear')(decoded)

    # This model maps an input to its reconstruction
    autoencoder = Model(input_data, decoded)

    # This model maps an input to its encoded representation
    # encoder = Model(input_data, encoded)

    # Compile the model
    autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

    # Display the architecture of the autoencoder
    # with lock:
    #     autoencoder.summary()

    # Train the autoencoder
    # We will use new_csv_features as both the input and the target since it's an autoencoder

    early_stopping = EarlyStopping(monitor='val_loss', patience=40, verbose=1)

    # Train the autoencoder and save the history
    history = autoencoder.fit(input_tensor_features, output_tensor_features,
                            epochs=2048,
                            batch_size=16,
                            shuffle=True,
                            validation_split=0.2,
                            callbacks=[early_stopping],
                            verbose=0,
                            )  # Set verbose to 1 for progress output




    best_loss = min(history.history['val_loss']) # this is the raining loss
    with lock:
        try:
            best_losses[encoding_dim] = min(best_loss,best_losses[encoding_dim])
        except:
            best_losses[encoding_dim] = best_loss
            
    print("latent dim:", encoding_dim, "loss value is: ", best_loss)

    if plot:
        lock.acquire()
        plt.figure(figsize=(10, 2),dpi=80)
        plt.plot(output_tensor_features[199], label='original')
        plt.plot(autoencoder(input_tensor_features)[199], label='reconstructed')
        plt.title('Latnet dim:'+ str(encoding_dim)+", best loss:"+str(best_loss)) 
        plt.legend()
        plt.show()
        plt.close()

        # plt.figure(figsize=(20, 4))
        # for i in range (1,len(autoencoder.layers)-1):
        # # Assuming 'autoencoder' is your trained model and 'layer_index' is the index of the layer you want to analyze
        #     weights = autoencoder.layers[i].get_weights()[0]  # 0 for weights, 1 for biases

        #     # Plotting the weights as a heatmap
            
        #     plt.subplot(1,len(autoencoder.layers)-2,i)
        #     plt.imshow(weights, cmap='viridis', aspect='auto')
        #     plt.colorbar()

        # plt.show()
        # plt.close()

        # plt.figure(figsize=(10, 16))
        # for i in range (1,len(autoencoder.layers)-1):
        # # Assuming 'autoencoder' is your trained model and 'layer_index' is the index of the layer you want to analyze
        #     weights = autoencoder.layers[i].get_weights()[0]  # 0 for weights, 1 for biases
        #     biases = autoencoder.layers[i].get_weights()[1]  # 0 for weights, 1 for biases
        #     # Histogram of the weight values
        #     plt.subplot(2*(len(autoencoder.layers)-2),1,i*2-1)
        #     plt.plot(weights.flatten().tolist(), label='weights')
        #     plt.plot(np.linspace(0,len(weights.flatten()),len(biases.flatten())),biases.flatten().tolist(), label='biases')
        #     plt.legend()
        #     plt.subplot(2*(len(autoencoder.layers)-2),2,i*4-1)
        #     plt.hist(weights.flatten(), bins=50)

            
        #     # Histogram of the biases values
        #     plt.subplot(2*(len(autoencoder.layers)-2),2,i*4)
        #     plt.hist(biases.flatten(), bins=50)

        # plt.show()
        # plt.close()

        lock.release()
    tf.keras.backend.clear_session()
    clear_session()




# Create a StandardScaler object
scaler = StandardScaler()
scaler2 = MinMaxScaler(feature_range=(0, 1))
# Assuming 'new_csv_features' is your data
def normalized(row):
    row = np.array(row)
    return scaler2.fit_transform(scaler.fit_transform(row.reshape(-1, 1)))

def fft_normalized(row):
    row = np.array(row)
    temp = np.fft.fft(row)[0:len(row)//2 + 1]

    amp = normalized(np.abs(temp))
    filtering = amp > np.average(amp)*.05
    angle = normalized(np.angle(temp))

    angle = angle[filtering] 
    angle = np.concatenate([angle,np.zeros(len(row)-len(angle))])
    amp = amp[filtering] 
    amp = np.concatenate([amp,np.zeros(len(row)-len(amp))])
    
    return np.concatenate([amp,angle])

lock = threading.Lock()


filtering = {''}
df = query(BLE['onBody'], {'pos':'static','antenna_side':'left'})
min_length = df['frame'].apply(len).min()
df['frame'] = df['frame'].apply(lambda x: x[:2000])
print(len(df['frame'][0]))
print(type(df['frame'][0]))

iq = IQ.IQ(Fc=2439810000+.1e4)

def configCreator(downSampleRate = 1, cutoff = 1e6):
    downSampleRate= max(downSampleRate, 1)
    return {                                      
            # iq.gradient:{},
            iq.unwrapPhase:{},
            iq.phase:{}, 
            # iq.butter:{'Fs': iq.Fs/downSampleRate, "cutoff": cutoff},
            iq.downSample:{'downSampleRate':downSampleRate, "shift": 0},
            iq.demodulate:{'Fs': iq.Fs},
           } 

methods = configCreator(downSampleRate=  4)
df['data'] = iq.apply(methods = methods, frame = df)
####################################################################################
# df['normalized'] = df['predictor'].apply(lambda x: normalized(x))

df['normalized'] = df['data'].apply(lambda x: fft_normalized(x))
input_tensor_features = tf.convert_to_tensor(df['normalized'].tolist())

df['normalized'] = df['data']#.apply(lambda x: normalized(x))
output_tensor_features = tf.convert_to_tensor(df['normalized'].tolist())

data_shape = len(df['data'][0])
#####################################################################################

best_losses = {}
batch_size = 3
for batch in range(0,int(np.ceil(data_shape/batch_size))):
    # try:
    #     run(batch,output_tensor_features,output_tensor_features,True)
    # except:
    #     continue
    threads = []
    # for i in range(batch_size * batch+1, min(batch_size * batch+batch_size+1,data_shape)):
    for i in range(batch_size): # runnig the code batch_size times in parallel to find the miunimum loss
        threads.append(threading.Thread(target=run, args=(batch+1,output_tensor_features,output_tensor_features,False, )))
        threads[-1].start()
    for t in threads:
        t.join()

best_losses = dict(sorted(best_losses.items()))
lossDF = pd.DataFrame(best_losses.values(),index=best_losses.keys())
lossDF.plot()
lossDF.to_csv('loss.csv')
plt.xlabel('Latent dimension')
plt.ylabel('Best loss')
plt.title('dataset: onBody static left')
plt.savefig('res/'+"onBody static left"+'.png')
plt.show()
plt.close()

2023-11-16 00:10:35.135861: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-16 00:10:35.159166: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-16 00:10:35.159189: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-16 00:10:35.159207: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-16 00:10:35.163858: I tensorflow/core/platform/cpu_feature_g

KeyboardInterrupt: 

In [ ]:
best_losses

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

row =np.random.rand(10) 

filename = files[3]
print(filename)
f = open(dataDir+filename, 'rb')
data_tuples = pickle.load(f)
f.close()

df = pd.DataFrame(data_tuples, columns=['predictor', 'label'])

def fft_normalized(row):
    temp = np.fft.fft(row)[0:len(row)//2 + 1]
    amp = normalized(np.abs(temp))
    filtering = amp > np.max(amp)*0.05 

    angle = normalized(np.angle(temp))
    angle = angle[filtering]
    amp = amp[filtering]

    return np.concatenate([amp,angle])


df['normalized'] = df['predictor'].apply(lambda x: fft_normalized(x))
tf.convert_to_tensor( df['normalized'].to_list())


In [ ]:
# best_losses = dict(sorted(best_losses.items()))
# lossDF = pd.DataFrame(best_losses.values(),index=best_losses.keys())

def read_loss(file :str | os.DirEntry):
    if type(file) is str:
        if file.endswith(".csv"):
            lossDF = pd.read_csv(file,index_col=0)
            return lossDF
    else:
        if file.name.endswith(".csv"):
            lossDF = pd.read_csv(file.path,index_col=0)
            return lossDF

def plot_loss(file):
        read_loss(file)
        lossDF.plot()
        plt.xlabel('Latent dimension')
        plt.ylabel('Best loss')
        plt.title('dataset: '+file.name)
        plt.savefig('res/'+file.name+'.png')
        plt.show()
        plt.close()

# for file in os.scandir('res/RAE-_l2-0.001/'):
df = read_loss("loss_dataset1.pkl.csv")
import scipy.signal as signal

def smooth(x,window_len=11,window='hanning'):
    if x.ndim != 1:
        raise ValueError("smooth only accepts 1 dimension arrays.")
    if x.size < window_len:
        raise ValueError("Input vector needs to be bigger than window size.")
    if window_len<3:
        return x
    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError(f"Window is on of '{window}'")
    s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    #print(len(s))
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval(f'np.{window}({window_len})')
    y=np.convolve(w/w.sum(),s,mode='valid')
    return y

plt.plot(smooth(df['0'].to_numpy(),window_len=5, window='hamming'))

